In [35]:
import pandas as pd
from sklearn.model_selection import train_test_split
from spectrograms import *
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, concatenate, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
import pandas as pd

### Tests on how to train a CNN using the data we have 775*308px

In [36]:
df = pd.read_csv('birdclef-2024/train_metadata.csv')
df['filepath'] = 'birdclef-2024/train_audio/' + df.filename
df['target'] = df.primary_label.map(CFG.name2label)
df['filename'] = df.filepath.map(lambda x: x.split('/')[-1])
df['xc_id'] = df.filepath.map(lambda x: x.split('/')[-1].split('.')[0])

train_df, test_df = train_test_split(df, test_size=0.3, random_state=42)

In [37]:
train_df.head(3)

,primary_label,secondary_labels,type,latitude,longitude,scientific_name,common_name,author,license,rating,url,filename,filepath,target,xc_id
8932,eaywag1,[],[''],43.3479,4.7934,Motacilla flava,Western Yellow Wagtail,Stanislas Wroza,Creative Commons Attribution-NonCommercial-Sha...,2.0,https://xeno-canto.org/790724,XC790724.ogg,birdclef-2024/train_audio/eaywag1/XC790724.ogg,53,XC790724
7578,comsan,[],['nocturnal flight call'],43.5352,-1.4475,Actitis hypoleucos,Common Sandpiper,Frédéric Cazaban,Creative Commons Attribution-NonCommercial-Sha...,5.0,https://www.xeno-canto.org/644516,XC644516.ogg,birdclef-2024/train_audio/comsan/XC644516.ogg,44,XC644516
1918,bkskit1,[],"['call', 'female', 'flight call', 'male']",37.3823,-6.1363,Elanus caeruleus,Black-winged Kite,José Carlos Sires,Creative Commons Attribution-NonCommercial-Sha...,5.0,https://www.xeno-canto.org/353823,XC353823.ogg,birdclef-2024/train_audio/bkskit1/XC353823.ogg,13,XC353823


In [38]:
df["target"].value_counts()

target
181    500
105    500
107    500
37     500
39     500
      ... 
16       6
178      6
119      5
5        5
92       5
Name: count, Length: 182, dtype: int64

In [39]:
target_counts = df['target'].value_counts()
print(len(target_counts[target_counts == 500]))

22


In [40]:
def create_model(input_shape_img, input_shape_meta, num_classes):
    # Image input branch
    img_input = Input(shape=input_shape_img, name='img_input')
    
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(img_input)
    x = MaxPooling2D((2, 2))(x)
    x = BatchNormalization()(x)
    
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2))(x)
    x = BatchNormalization()(x)
    
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2))(x)
    x = BatchNormalization()(x)
    
    x = Flatten()(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    
    # Metadata input branch
    meta_input = Input(shape=input_shape_meta, name='meta_input')
    y = Dense(32, activation='relu')(meta_input)
    
    # Combine both branches
    combined = concatenate([x, y])
    z = Dense(128, activation='relu')(combined)
    z = Dropout(0.5)(z)
    z = Dense(num_classes, activation='softmax')(z)
    
    model = Model(inputs=[img_input, meta_input], outputs=z)
    
    return model

In [41]:
input_shape_img_rgb = (*CFG.img_size, 3)  # for RGB images
input_shape_img_bw = (*CFG.img_size, 1)  # for grayscale images
input_shape_meta = (2,)  # for latitude and longitude
num_classes = CFG.num_classes

In [42]:
model = create_model(input_shape_img_rgb, input_shape_meta, num_classes)
model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
train_generator = data_generator(train_df, batch_size=CFG.batch_size, rgb=False)

In [43]:
# Verify the generator
batch_size = CFG.batch_size
test_generator = data_generator(train_df, batch_size, img_size=CFG.img_size, rgb=False)

for _ in range(1):
    [img_batch, meta_batch], target_batch = next(test_generator)
    print(f'Image batch shape: {img_batch.shape}')
    print(f'Metadata batch shape: {meta_batch.shape}')
    print(f'Target batch shape: {target_batch.shape}')
    break


TypeError: data_generator() got an unexpected keyword argument 'img_size'

In [44]:
model.fit(train_generator, steps_per_epoch=len(train_df) // CFG.batch_size, epochs=CFG.epochs)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 2 dimensions. The detected shape was (64, 256) + inhomogeneous part.